In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
import os

IMAGE_FILE = "../output/std_images.npy"  
VELOCITY_FILE = "../output/Town06_500s-3_velocity_Smaller.npy"
BATCH_SIZE = 32
NUM_EPOCHS = 10
LEARNING_RATE = 1e-4

#Dataset
class VelocityDataset(Dataset):
    def __init__(self, image_path, velocity_path, transform=None):
        self.images = np.load(image_path)
        self.velocities = np.load(velocity_path)

        if self.images.ndim == 4 and self.images.shape[1] == 3:
            self.images = np.transpose(self.images, (0, 2, 3, 1))

        if self.images.max() <= 1.0:
            self.images = (self.images * 255).astype(np.uint8)

        self.transform = transform

    def __len__(self):
        return len(self.velocities)

    def __getitem__(self, idx):
        img = Image.fromarray(self.images[idx])
        velocity = self.velocities[idx]

        if self.transform:
            img = self.transform(img)

        return img, torch.tensor([velocity], dtype=torch.float32)

#Image Transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

class VelocityRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        base = models.resnet18(pretrained=True)
        #Regression output
        base.fc = nn.Linear(base.fc.in_features, 1)  
        self.model = base

    def forward(self, x):
        return self.model(x)

dataset = VelocityDataset(IMAGE_FILE, VELOCITY_FILE, transform)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VelocityRegressor().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    for images, targets in dataloader:
        images, targets = images.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    avg_loss = running_loss / len(dataloader.dataset)
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} - Loss: {avg_loss:.4f}")

torch.save(model.state_dict(), "velocity_regressor.pth")
print("Model saved as velocity_regressor.pth")